In [1]:
%load_ext autoreload
%autoreload 2

# Hexagon Regression


In [2]:
# ruff: noqa: E402

import os
import sys
from pathlib import Path

# add the
ROOT = Path(os.getcwd())
while not (ROOT / ".git").exists():
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

from src.config import load_config
from src.models.dataset import ServiceTimeDataset
from src.models.xgboostlss import XGBoostLSSModel
from src.models.conformal import ConformalModel
import polars as pl
import geopandas as gpd
import pandas as pd

## Tune XGBoostLSS


In [3]:
# import multiprocessing as mp

# ncpus = mp.cpu_count()

# dtrain = xgb.DMatrix(
#     dataset.train_feature_array,
#     label=dataset.train_label_array,
#     nthread=ncpus,
# )

# dtest = xgb.DMatrix(
#     dataset.test_feature_array,
#     nthread=ncpus,
# )

In [4]:
# np.random.seed(123)

# opt_params = opt_param.copy()
# n_rounds = opt_params["opt_rounds"]
# del opt_params["opt_rounds"]

# opt_param

In [5]:
opt_params = {
    "eta": 0.04745083489010809,
    "max_depth": 2,
    "gamma": 0.014864286418162793,
    "subsample": 0.5428946911410379,
    "colsample_bytree": 0.6767232365345294,
    "min_child_weight": 9.02642750608808,
    "booster": "gbtree",
    "lambda": 6.16767924247347e-08,
    "objective": None,
    "base_score": 0,
    "num_target": 2,
    "disable_default_eval_metric": True,
}

# opt_params = {
#     "eta": 0.1770036172761959,
#     "max_depth": 7,
#     "gamma": 1.6967679871536873e-08,
#     "subsample": 0.9627038731587274,
#     "colsample_bytree": 0.8355652183200688,
#     "min_child_weight": 1.2004128119820168e-05,
#     "booster": "dart",
#     "lambda": 0.0978870405389456,
#     "opt_rounds": 2,
# }

## Testing a Bunch of Models


In [6]:
# from sklearn.metrics import r2_score
# from itertools import product
# from src.models.xgboostlss import XGBoostLSSModel
# from tqdm import tqdm

# # get all the supported distributions
# import xgboostlss.distributions as dists


# dataset = ServiceTimeDataset(
#     config=load_config(ROOT / "config" / "paper.yaml"),
# )

# scores = []


# for dist_cls, loss_fn, stabilization, response_function, target, city in tqdm(
#     list(
#         product(
#             [
#                 dists.Weibull.Weibull,
#                 dists.LogNormal.LogNormal,
#                 dists.Gaussian.Gaussian,
#                 dists.StudentT.StudentT,
#             ],
#             ["nll", "crps"],
#             ["MAD", "L2"],
#             ["exp", "softplus"],
#             ["service_time", "log_service_time"],
#             [
#                 "Boston, USA",
#                 "Seattle, USA",
#             ],
#         )
#     )
# ):
#     for i, kf_dataset in enumerate(
#         dataset.kfold_split(
#             cities=[city],
#             min_points_per_hex=10,
#             k=5,
#             seed=42,
#         )
#     ):
#         try:
#             xgblss = XGBoostLSSModel(
#                 xgboost_params={},
#                 distribution=dist_cls(
#                     loss_fn=loss_fn,
#                     stabilization=stabilization,
#                     response_fn=response_function,
#                 ),
#             )
#             kf_dataset = kf_dataset.set_pred_col(target).apply_model(
#                 xgblss,
#                 num_boost_round=300,
#             )

#             score_df = kf_dataset.score_test(
#                 models=[xgblss],
#                 transform_log=True,
#             ).with_columns(
#                 city=pl.lit(city),
#                 loss_fn=pl.lit(loss_fn),
#                 stabilization=pl.lit(stabilization),
#                 response_function=pl.lit(response_function),
#                 target=pl.lit(target),
#                 distribution=pl.lit(dist_cls.__name__),
#                 k_fold=pl.lit(i),
#             )

#             scores.append(score_df)
#         except Exception as e:
#             print(e)
#             continue

In [7]:
# score_df = (
#     pl.concat(scores)
#     .group_by(
#         ["loss_fn", "stabilization", "response_function", "target", "distribution"]
#     )
#     .agg(pl.all().mean())
#     .with_columns(
#         # pl.concat_list(pl.col("^r2.*$")).list.mean().alias("r_mean"),
#         pl.concat_list(pl.col("^pinball.*$")).list.mean().alias("pinball_mean"),
#         pl.concat_list(pl.col("^rmse.*$")).list.mean().alias("rmse_mean"),
#     )
#     .with_columns(
#         # pl.col("r_mean").rank().alias("r_rank"),
#         pl.col("pinball_mean").rank().alias("pinball_rank"),
#         pl.col("crps").rank().alias("crps_rank"),
#         pl.col("rmse_mean").rank().alias("rmse_rank"),
#     )
#     .with_columns(
#         (pl.col("crps_rank") + pl.col("pinball_rank") + pl.col("rmse_rank")).alias(
#             "total_rank"
#         )
#     )
#     .sort("total_rank")
# )

# score_df.write_csv(
#     ROOT / "data" / "model_results" / "xgboostlss_distribution_comparison.csv"
# )

In [8]:
import polars as pl

score_df = pl.read_csv(
    ROOT / "data" / "model_results" / "xgboostlss_distribution_comparison.csv"
)

(
    score_df.with_columns(
        (pl.col("crps_rank") + pl.col("pinball_rank")).alias("total_rank")
    ).sort("total_rank")
)

loss_fn,stabilization,response_function,target,distribution,model,crps,mean_coverage,mean_coverage_std,mean_interval_width,pinball_0.05,mae_0.05,rmse_0.05,pinball_0.5,mae_0.5,rmse_0.5,pinball_0.95,mae_0.95,rmse_0.95,city,k_fold,pinball_mean,rmse_mean,pinball_rank,crps_rank,rmse_rank,total_rank
str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""nll""","""L2""","""softplus""","""service_time""","""LogNormal""",null,75.651683,0.8827,0.0845,350.715205,7.022662,132.887705,235.575526,48.827256,97.654512,194.922784,28.191918,267.267573,321.988565,null,2.0,28.013945,250.828958,2.0,1.0,14.0,3.0
"""nll""","""MAD""","""softplus""","""log_service_ti…","""LogNormal""",null,75.898618,0.885349,0.079942,383.164714,6.998826,130.442853,233.793719,48.716993,97.433986,196.969315,28.210003,300.708288,360.625807,null,2.0,27.975274,263.79628,1.0,5.0,39.0,6.0
"""nll""","""L2""","""exp""","""log_service_ti…","""Gaussian""",null,75.780195,0.883754,0.085074,349.05383,7.054223,133.216134,235.596596,49.012231,98.024462,195.187306,28.318516,265.017654,316.950474,null,2.0,28.128324,249.244792,6.0,3.0,9.0,9.0
"""nll""","""MAD""","""exp""","""log_service_ti…","""LogNormal""",null,76.0128,0.8848,0.083849,385.108777,7.028092,130.666222,234.230388,48.775322,97.550645,197.26895,28.288458,306.587039,366.734243,null,2.0,28.030624,266.07786,3.0,7.0,41.0,10.0
"""nll""","""L2""","""softplus""","""log_service_ti…","""Gaussian""",null,75.99773,0.883731,0.084687,349.912125,7.049991,133.837616,236.925157,48.999667,97.999334,195.882006,28.366857,268.282386,323.485426,null,2.0,28.138838,252.09753,7.0,6.0,17.0,13.0
"""crps""","""L2""","""exp""","""log_service_ti…","""Gaussian""",null,75.85958,0.866352,0.086778,321.405092,7.019353,132.503468,235.816436,48.849867,97.699733,196.321877,29.067233,241.248228,296.860953,null,2.0,28.312151,242.999755,16.0,4.0,4.0,20.0
"""crps""","""L2""","""softplus""","""log_service_ti…","""StudentT""",null,75.747457,0.870702,0.086499,322.979244,7.029432,132.885448,237.001108,48.875384,97.750767,197.402926,29.071359,242.402406,297.915706,null,2.0,28.325391,244.10658,18.0,2.0,6.0,20.0
"""nll""","""L2""","""exp""","""service_time""","""LogNormal""",null,76.097859,0.881892,0.086586,346.929154,7.062826,133.98028,237.233163,49.063046,98.126092,196.585569,28.619061,265.325686,320.379808,null,2.0,28.248311,251.399513,11.0,10.0,16.0,21.0
"""nll""","""MAD""","""exp""","""log_service_ti…","""Gaussian""",null,76.225711,0.892075,0.081777,353.233498,7.061836,135.479394,238.935645,49.108194,98.216388,198.364377,28.445882,268.391754,313.703246,null,2.0,28.205304,250.334422,9.0,14.0,11.0,23.0


In [9]:
# print the best model
print(
    score_df[
        ["loss_fn", "stabilization", "response_function", "target", "distribution"]
    ][0]
)

shape: (1, 5)
┌─────────┬───────────────┬───────────────────┬──────────────┬──────────────┐
│ loss_fn ┆ stabilization ┆ response_function ┆ target       ┆ distribution │
│ ---     ┆ ---           ┆ ---               ┆ ---          ┆ ---          │
│ str     ┆ str           ┆ str               ┆ str          ┆ str          │
╞═════════╪═══════════════╪═══════════════════╪══════════════╪══════════════╡
│ nll     ┆ L2            ┆ softplus          ┆ service_time ┆ LogNormal    │
└─────────┴───────────────┴───────────────────┴──────────────┴──────────────┘


## Optimizing the Model


In [10]:
from sklearn.model_selection import KFold



def get_kfolds(dataset, nfolds):
    regions = dataset.train_df['region_id'].unique()



    kfold_indices = KFold(n_splits=nfolds, shuffle=False, ).split(
        regions
    )

    dataset_inds = []

    for i, (train_index, test_index) in enumerate(kfold_indices):
        train_regions = regions[train_index]
        test_regions = regions[test_index]

        train_idx = dataset.train_df.with_row_index().filter(
            pl.col("region_id").is_in(train_regions)
        )['index'].to_list()
        test_idx = dataset.train_df.with_row_index().filter(
            pl.col("region_id").is_in(test_regions)
        )['index'].to_list()

        dataset_inds.append((train_idx, test_idx))

    return dataset_inds

In [14]:
import xgboost
from xgboostlss.model import XGBoostLSS
import numpy as np
import multiprocessing as mp
import xgboostlss.distributions as dists


xgblss = XGBoostLSS(
    dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="None",
        response_fn="softplus",
    ),
)

# make the dataset
dataset = ServiceTimeDataset(
    config=load_config(ROOT / "config" / "paper.yaml"),
).split_test_train(
    train_cities=["Boston, USA"],
    test_cities=["Boston, USA"],
    min_points_per_hex_test=1,
    min_points_per_hex_train=1,
    seed=42,
    train_size=0.99,
)


kfold_indices = get_kfolds(dataset, 5)


dtrain = xgboost.DMatrix(
    dataset.train_feature_array,
    label=dataset.train_label_array,
    nthread=mp.cpu_count(),
)

param_dict = {
    "eta": ["float", {"low": 1e-4, "high": 1, "log": True}],
    "max_depth": ["int", {"low": 4, "high": 15, "log": False}],
    "gamma": ["float", {"low": 1e-8, "high": 40, "log": True}],
    "subsample": ["float", {"low": 0.7, "high": 1.0, "log": False}],
    "colsample_bytree": ["float", {"low": 0.8, "high": 1.0, "log": False}],
    "colsample_bylevel": ["float", {"low": 0.8, "high": 1.0, "log": False}],
    "min_child_weight": ["float", {"low": 1e-8, "high": 500, "log": True}],
    "booster": [
        "categorical",
        [
            "gbtree",
            # "gblinear"
            # "dart",
        ],
    ],
    "tree_method": [
        "categorical",
        [
            "exact",
            "approx",
            "hist",
        ],
    ],
    "reg_lambda": ["float", {"low": 1e-8, "high": 10, "log": True}],
}


np.random.seed(123)
opt_param = xgblss.hyper_opt(
    param_dict,
    dtrain,
    num_boost_round=500,  # Number of boosting iterations.
    nfold=5,  # Number of cv-folds.
    folds=kfold_indices,
    early_stopping_rounds=100,  # Number of early-stopping rounds
    max_minutes=30,  # Time budget in minutes, i.e., stop study after the given number of minutes.
    n_trials=None,  # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
    silence=False,  # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
    seed=123,  # Seed used to generate cv-folds.
    hp_seed=None,  # Seed for random number generator used in the Bayesian hyperparameter search.
)

[I 2024-01-31 10:31:35,096] A new study created in memory with name: XGBoostLSS Hyper-Parameter Optimization


   0%|          | 00:00/30:00

[I 2024-01-31 10:31:52,816] Trial 0 finished with value: 26373.583203 and parameters: {'eta': 0.01663806263479828, 'max_depth': 13, 'gamma': 9.853404916451053e-07, 'subsample': 0.9967960591213362, 'colsample_bytree': 0.9124053178315806, 'colsample_bylevel': 0.9489617459075623, 'min_child_weight': 2.729774053332432e-07, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 0.00019788908070097816}. Best is trial 0 with value: 26373.583203.
[I 2024-01-31 10:32:29,008] Trial 1 finished with value: 25833.8007812 and parameters: {'eta': 0.004408422642897559, 'max_depth': 9, 'gamma': 0.00030854972924565716, 'subsample': 0.9034912128489402, 'colsample_bytree': 0.9324182537129015, 'colsample_bylevel': 0.8282467598524368, 'min_child_weight': 2.8335921108523192e-08, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 0.05674360576161859}. Best is trial 1 with value: 25833.8007812.


In [13]:
from sklearn.model_selection import KFold
import xgboost
from xgboostlss.model import XGBoostLSS
import numpy as np
import multiprocessing as mp

from src.osm_tags import build_tag_filter

config = load_config(ROOT / "config" / "paper.yaml")
target_tags = build_tag_filter(config)
target_tag_list = [f"{t}_{st}" for t, sts in target_tags.items() for st in sts]
super_tags = list(target_tags.keys())


xgblss = XGBoostLSS(
    dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="None",
        response_fn="softplus",
    ),
)

# make the dataset
dataset = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_tags()
    .split_test_train(
        train_cities=["Boston, USA"],
        test_cities=["Boston, USA"],
        min_points_per_hex_test=5,
        min_points_per_hex_train=5,
        seed=42,
        train_size=0.99,
    )
    .set_feature_cols(target_tag_list)
)

kfold_indices = get_kfolds(dataset, 5)

dtrain = xgboost.DMatrix(
    dataset.train_feature_array,
    label=dataset.train_label_array,
    nthread=mp.cpu_count(),
)

param_dict = {
    # "eta": ["float", {"low": 1e-5, "high": 1, "log": True}],
    "max_depth": ["int", {"low": 1, "high": 8, "log": False}],
    "gamma": ["float", {"low": 1e-8, "high": 40, "log": True}],
    "subsample": ["float", {"low": 0.7, "high": 1.0, "log": False}],
    "colsample_bytree": ["float", {"low": 0.5, "high": 1.0, "log": False}],
    "colsample_bylevel": ["float", {"low": 0.5, "high": 1.0, "log": False}],
    "min_child_weight": ["float", {"low": 1e-8, "high": 500, "log": True}],
    "booster": [
        "categorical",
        [
            "gbtree",
            # "dart",
        ],
    ],
    "tree_method": [
        "categorical",
        [
            "exact",
            "approx",
            "hist",
        ],
    ],
    "reg_lambda": ["float", {"low": 1e-8, "high": 10, "log": True}],
}


np.random.seed(123)
opt_param = xgblss.hyper_opt(
    param_dict,
    dtrain,
    num_boost_round=500,  # Number of boosting iterations.
    nfold=5,  # Number of cv-folds.
    folds=kfold_indices,
    early_stopping_rounds=20,  # Number of early-stopping rounds
    max_minutes=30,  # Time budget in minutes, i.e., stop study after the given number of minutes.
    n_trials=None,  # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
    silence=False,  # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
    seed=123,  # Seed used to generate cv-folds.
    hp_seed=None,  # Seed for random number generator used in the Bayesian hyperparameter search.
)

[I 2024-01-31 09:10:34,410] A new study created in memory with name: XGBoostLSS Hyper-Parameter Optimization


   0%|          | 00:00/30:00

[I 2024-01-31 09:10:59,116] Trial 0 finished with value: 25361.132030999997 and parameters: {'max_depth': 1, 'gamma': 0.0020677124205099044, 'subsample': 0.8593887054392787, 'colsample_bytree': 0.5777381314321288, 'colsample_bylevel': 0.7804877392551476, 'min_child_weight': 15.398618663649936, 'booster': 'gbtree', 'tree_method': 'approx', 'reg_lambda': 0.02192371613444977}. Best is trial 0 with value: 25361.132030999997.
[I 2024-01-31 09:11:25,682] Trial 1 finished with value: 27824.813281400002 and parameters: {'max_depth': 7, 'gamma': 6.1913795173891945e-06, 'subsample': 0.7321109013668997, 'colsample_bytree': 0.7446520448356292, 'colsample_bylevel': 0.5231393591606217, 'min_child_weight': 0.001660865375898693, 'booster': 'gbtree', 'tree_method': 'exact', 'reg_lambda': 9.933088522724713e-08}. Best is trial 0 with value: 25361.132030999997.
[I 2024-01-31 09:12:18,696] Trial 2 finished with value: 25521.8113282 and parameters: {'max_depth': 8, 'gamma': 23.66699752813613, 'subsample': 0

KeyboardInterrupt: 

In [41]:
import xgboost
from xgboostlss.model import XGBoostLSS
import numpy as np
import multiprocessing as mp

from src.osm_tags import build_tag_filter

config = load_config(ROOT / "config" / "paper.yaml")
target_tags = build_tag_filter(config)
target_tag_list = [f"{t}_{st}" for t, sts in target_tags.items() for st in sts]
super_tags = list(target_tags.keys())


xgblss = XGBoostLSS(
    dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="None",
        response_fn="softplus",
    ),
)

# make the dataset
dataset = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_tags()
    .split_test_train(
        train_cities=["Boston, USA"],
        test_cities=["Boston, USA"],
        min_points_per_hex_test=1,
        min_points_per_hex_train=1,
        seed=42,
        train_size=0.99,
    )
    .set_feature_cols(target_tag_list)
)


dtrain = xgboost.DMatrix(
    dataset.train_feature_array,
    label=dataset.train_label_array,
    nthread=mp.cpu_count(),
)

param_dict = {
    "eta": ["float", {"low": 1e-5, "high": 1, "log": True}],
    "max_depth": ["int", {"low": 1, "high": 8, "log": False}],
    "gamma": ["float", {"low": 1e-8, "high": 40, "log": True}],
    "subsample": ["float", {"low": 0.7, "high": 1.0, "log": False}],
    "colsample_bytree": ["float", {"low": 0.5, "high": 1.0, "log": False}],
    "colsample_bylevel": ["float", {"low": 0.5, "high": 1.0, "log": False}],
    "min_child_weight": ["float", {"low": 1e-8, "high": 500, "log": True}],
    "booster": [
        "categorical",
        [
            "gbtree",
            "gbliner",
            # "dart",
        ],
    ],
    "tree_method": [
        "categorical",
        [
            "exact",
            "approx",
            "hist",
        ],
    ],
    "reg_lambda": ["float", {"low": 1e-8, "high": 10, "log": True}],
}


np.random.seed(123)
opt_param = xgblss.hyper_opt(
    param_dict,
    dtrain,
    num_boost_round=300,  # Number of boosting iterations.
    nfold=5,  # Number of cv-folds.
    early_stopping_rounds=20,  # Number of early-stopping rounds
    max_minutes=60,  # Time budget in minutes, i.e., stop study after the given number of minutes.
    n_trials=None,  # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
    silence=False,  # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
    seed=123,  # Seed used to generate cv-folds.
    hp_seed=None,  # Seed for random number generator used in the Bayesian hyperparameter search.
)

[autoreload of src.models.conformal failed: Traceback (most recent call last):
  File "/Users/max/.pyenv/versions/3.11.6/envs/cargo-bike-analysis-11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/max/.pyenv/versions/3.11.6/envs/cargo-bike-analysis-11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/max/.pyenv/versions/3.11.6/envs/cargo-bike-analysis-11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/max/.pyenv/versions/3.11.6/envs/cargo-bike-analysis-11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/max/.pyenv/versions/3.11.6/envs/cargo-bike-analysis-11/lib/python3.11/site-packages/IPython/

   0%|          | 00:00/30:00

[I 2024-01-28 19:19:38,580] Trial 0 finished with value: 20403.0546876 and parameters: {'max_depth': 1, 'gamma': 0.17320208591224567, 'subsample': 0.7996680441165032, 'colsample_bytree': 0.7279784080170321, 'colsample_bylevel': 0.9378616178544024, 'min_child_weight': 0.12203140349608772, 'booster': 'gbtree', 'tree_method': 'hist', 'reg_lambda': 8.494642219299579e-07}. Best is trial 0 with value: 20403.0546876.
[I 2024-01-28 19:20:09,767] Trial 1 finished with value: 20446.4257812 and parameters: {'max_depth': 1, 'gamma': 1.074155222366971e-05, 'subsample': 0.7381073118963637, 'colsample_bytree': 0.6595540137500423, 'colsample_bylevel': 0.8322761972917069, 'min_child_weight': 2.102248958685932, 'booster': 'gbtree', 'tree_method': 'exact', 'reg_lambda': 4.37497689090557}. Best is trial 0 with value: 20403.0546876.
[I 2024-01-28 19:21:21,566] Trial 2 finished with value: 20368.119531199998 and parameters: {'max_depth': 8, 'gamma': 9.914182115836651e-06, 'subsample': 0.8529905111686795, 'c

### Optimizing Learning Rate and Number of Trees


In [47]:
from itertools import product
from tqdm import tqdm

from src.models.xgboostlss import OPT_PARAMS


dataset = ServiceTimeDataset(
    config=load_config(ROOT / "config" / "paper.yaml"),
)

results = []


for lr, n_estimators in tqdm(
    list(
        product(
            [0.0001, 0.001, 0.01, 0.1],
            [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
        )
    )
):
    for kf_dataset in dataset.kfold_split(
        cities=["Boston, USA"],
        min_points_per_hex=10,
        k=5,
        seed=42,
    ):
        model = XGBoostLSSModel(
            xgboost_params={"eta": lr, **OPT_PARAMS},
        )

        kf_dataset.apply_model(
            model,
            num_boost_round=n_estimators,
        )

        results.append(
            kf_dataset.score_test(
                models=[model],
                transform_log=False,
            ).with_columns(
                lr=pl.lit(lr),
                n_estimators=pl.lit(n_estimators),
            )
        )

100%|██████████| 40/40 [15:04<00:00, 22.60s/it]


In [52]:
pl.concat(results).group_by(["lr", "n_estimators"]).agg(pl.all().mean()).sort("MAE_0.5")

lr,n_estimators,model,crps,mean_coverage,mean_coverage_std,mean_interval_width,MAE_0.5,pinball_0.05,pinball_0.5,pinball_0.95
f64,i32,str,f64,f64,f64,f64,f64,f64,f64,f64
0.001,500,null,79.491068,0.875431,0.085079,353.354517,127.558814,7.33568,51.310606,29.876473
0.001,200,null,79.082012,0.878028,0.080301,354.328487,128.016796,7.321123,51.062116,29.447668
0.1,600,null,79.004627,0.877214,0.082937,353.003399,128.04279,7.296511,50.915177,29.472422
0.1,400,null,78.681713,0.880094,0.08414,355.606871,128.572601,7.288852,50.800498,29.558219
0.01,100,null,79.349083,0.878845,0.085661,355.299882,129.124097,7.335087,51.069015,29.581788
0.001,400,null,79.744792,0.871828,0.085817,351.516051,129.334053,7.370388,51.397273,29.7736
0.0001,200,null,79.239884,0.87903,0.081821,358.078597,129.339247,7.321688,51.087523,29.460582
0.001,1000,null,79.39541,0.874933,0.088357,356.753505,129.614655,7.350964,51.302398,29.660211
0.0001,100,null,79.68629,0.879118,0.088295,356.668695,129.714108,7.340524,51.260089,29.77528


## Chose Test-Train Size


In [ ]:
results = []

dataset = ServiceTimeDataset(
    config=load_config(ROOT / "config" / "paper.yaml"),
)

for train_size, test_size, city in tqdm(
    list(
        product(
            range(0, 30, 5),
            range(0, 30, 5),
            [
                "Boston, USA",
                "Seattle, USA",
            ],
        )
    )
):
    # if train_size > test_size:
    #     continue

    for i in range(5):
        model = XGBoostLSSModel(
            distribution=dists.LogNormal.LogNormal(
                loss_fn="nll",
                stabilization="L2",
                response_fn="softplus",
            ),
        )

        res = (
            dataset.copy()
            .split_test_train(
                train_cities=["Boston, USA"],
                test_cities=["Boston, USA"],
                min_points_per_hex_test=test_size,
                min_points_per_hex_train=train_size,
                seed=np.random.randint(1000),
            )
            .apply_model(
                model,
                num_boost_round=300,
            )
            .score_test(
                models=[model],
                transform_log=False,
            )
        )

        results.append(
            res.with_columns(
                city=pl.lit(city),
                train_size=pl.lit(train_size),
                test_size=pl.lit(test_size),
                k_fold=pl.lit(i),
            )
        )

In [ ]:
pl.concat(results).group_by(["train_size", "test_size"]).agg(pl.all().mean()).sort(
    "mae_0.5"
).head(10)

## Test-Train on Each City


In [ ]:
from src.osm_tags import build_tag_filter
from src.models.baseline import CityWideAverageModelDataset, KRingModelDataset
from src.models.xgboostlss import XGBoostLSSModel
from src.models.conformal import ConformalModel
from src.models.dataset import ServiceTimeDataset
import xgboostlss.distributions as dists

config = load_config(ROOT / "config" / "paper.yaml")
target_tags = build_tag_filter(config)
target_tag_list = [f"{t}_{st}" for t, sts in target_tags.items() for st in sts]
super_tags = list(target_tags.keys())


dataset = (
    ServiceTimeDataset(
        config=config,
    )
    .add_super_tags()
    .add_tags()
    .split_test_train(
        test_cities=["Boston, USA"],
        train_cities=["Boston, USA"],
    )
)

In [ ]:
m_embeddings = XGBoostLSSModel(
    distribution=dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="L2",
        response_fn="softplus",
    ),
    # feature_cols=super_tags,
    xgboost_params={},
    name="xgboostlss_embeddings",
)

m_st = XGBoostLSSModel(
    distribution=dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="L2",
        response_fn="softplus",
    ),
    xgboost_params={},
    feature_cols=super_tags,
    name="xgboostlss_st",
)

m_sbt = XGBoostLSSModel(
    distribution=dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="L2",
        response_fn="softplus",
    ),
    xgboost_params={},
    feature_cols=target_tag_list,
    name="xgboostlss_sbt",
)

# embeddings + st
m_emb_st = XGBoostLSSModel(
    distribution=dists.LogNormal.LogNormal(
        loss_fn="nll",
        stabilization="L2",
        response_fn="softplus",
    ),
    xgboost_params={},
    feature_cols=super_tags + list(map(str, range(50))),
    name="xgboostlss_emb_st",
)

for m in [m_embeddings, m_st, m_sbt, m_emb_st]:
    dataset = dataset.apply_model(m, num_boost_round=500)

In [ ]:
dataset.score_test(
    models=[m_embeddings, m_st, m_sbt, m_emb_st],
    transform_log=False,
).to_pandas().sort_values("crps")

In [ ]:
# dataset.test_feature_array

In [ ]:
models = [
    XGBoostLSSModel(
        distribution=dists.LogNormal.LogNormal(
            loss_fn="nll",
            stabilization="L2",
            response_fn="softplus",
        ),
    ),
    CityWideAverageModelDataset(),
    KRingModelDataset(k=10),
    # KRingModelDataset(k=7, name="7-ring"),
    # ConformalModel(),
]

results = []

for city in ["Boston, USA", "Seattle, USA"]:
    for i, split_dataset in enumerate(
        dataset.kfold_split(
            cities=[city],
            min_points_per_hex=5,
            k=5,
            seed=42,
        )
    ):
        for model in models:
            split_dataset = split_dataset.apply_model(
                model,
            )

        results.append(
            split_dataset.score_test(
                models=models, transform_log=False, crps_n=100
            ).with_columns(
                city=pl.lit(city),
                k_fold=pl.lit(i),
            )
        )

In [ ]:
res_df = (
    pl.concat(results)
    .drop("mean_coverage_std", "k_fold")
    .group_by(["city", "model"])
    .agg(
        pl.all().mean(),
        pl.all().std().name.map(lambda x: f"{x}_std"),
    )
)

res_df = (
    res_df.select(
        *(
            pl.concat_str(
                pl.lit("$"),
                pl.concat_str(
                    [
                        pl.col(col).round(1).cast(str),
                        pl.col(f"{col}_std").round(1).cast(str),
                    ],
                    separator="\pm",
                ),
                pl.lit("$"),
            ).alias(col)
            for col in res_df.columns
            if (
                col
                not in [
                    "city",
                    "model",
                ]
            )
            and ("std" not in col)
        ),
        "city",
        "model",
    )
    .rename({"mean_coverage": "coverage", "mean_interval_width": "interval_width"})
    .to_pandas()
    .set_index(["city", "model"])
)


# res_df = res_df.


def pretty_col(name, percent=None, *args, **kwargs):
    if "coverage" in name:
        return ("Coverage (%)",)
    elif "interval" in name:
        return ("Interval Width (s)",)
    elif "crps" in name:
        return ("CRPS (s)",)
    else:
        return (
            f"{name.upper()} (S)",
            "$P_{" + f"{int(float(percent) * 100):02d}" + "}$",
        )


# create a multiindex for the columns
res_df.columns = pd.MultiIndex.from_tuples(
    list(map(lambda x: tuple(pretty_col(*x.split("_"))), res_df.columns))
)

res_df.sort_index().sort_index(axis=1).head(10)

In [ ]:
# print(
print(
    res_df.sort_index(axis=1)
    .sort_index(axis=0)
    .to_latex(
        escape=False,
        # multirow=True,
        multicolumn_format="c",
        # column_format="l",
        float_format="{:0.2f}",
        na_rep="-",
        # highlight the min
        # highlight_min=True,
    )
)
# )

## Testing New Api


### Train Test Boston


In [ ]:
from src.models.baseline import CityWideAverageModelDataset, KRingModelDataset
from src.models.conformal import ConformalXGBoostModel
from xgboostlss.distributions import StudentT
import random


xgb_model = XGBoostLSSModel(
    distribution=StudentT.StudentT(
        # loss_fn="crps", response_fn="softplus", stabilization="L2"
    )
)
city_avg_model = CityWideAverageModelDataset()
k_ring_model = KRingModelDataset(k=7)
k_ring_model_3 = KRingModelDataset(k=3, name="k_ring_3")
conformal_model = ConformalXGBoostModel(cps=True)


applied_df = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    # .add_van_label()
    .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=["Boston, USA"],
        test_cities=["Boston, USA"],
        min_points_per_hex_train=10,
        min_points_per_hex_test=20,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
    )
    .apply_model(conformal_model)
    .apply_model(xgb_model)
    .apply_model(city_avg_model)
    .apply_model(k_ring_model)
    .apply_model(k_ring_model_3)
)

In [ ]:
applied_df.score_test(
    models=[conformal_model, xgb_model, city_avg_model, k_ring_model, k_ring_model_3],
    transform_log=True,
)

In [ ]:
# conformal_model._model.predict_cps(
#     applied_df.test_feature_array,
#     lower_percentiles=[0.05, ],
#     # higher_percentiles=[0.95],
#     # bins=100,
#     # return_cpds=True,
#     # cpds_by_bins=True,
# )

In [ ]:
conformal_model._model.predict_cps(
    applied_df.test_feature_array,
    lower_percentiles=[0.05, 0.5, 0.95],
    # higher_percentiles=[0.95],
)

In [ ]:
score_df = applied_df.score_test(
    models=[xgb_model, city_avg_model, k_ring_model, k_ring_model_3], crps_n=50
)

score_df.to_pandas().sort_index(
    axis=1,
).set_index("model")

In [ ]:
# # score the median prediction
# from sklearn.metrics import r2_score

# region_test_df = applied_df.test_df.group_by("region_id").agg(pl.all().first())


# for model in [
#     "XGBoostLSS",
#     "CityWideAverageModelDataset",
#     "KRingModelDataset",
#     "KRingModelDataset3",
# ]:
#     for q in [0.05, 0.5, 0.95]:
#         print(
#             model,
#             q,
#             round(
#                 r2_score(
#                     region_test_df["log_service_time_q{}".format(q)].exp(),
#                     region_test_df[f"{model}_quant_{q}"].exp(),
#                 ),
#                 3,
#             ),
#         )

In [ ]:
# import scienceplots
import seaborn as sns
import matplotlib.pyplot as plt

# plt.style.use(["science",])
test_df = pl.concat(
    (
        *(
            applied_df.test_df.select(
                [
                    (
                        pl.col(f"{model}_samples")
                        .exp()
                        .cast(pl.Float32)
                        .alias("service_time")
                        if model == "XGBoostLSS"
                        else pl.col(f"{model}_samples")
                        .cast(pl.Float32)
                        .alias("service_time")
                    ),
                    pl.col("cluster"),
                    pl.lit(model).alias("mode"),
                ]
            )
            for model in [
                "XGBoostLSS",
                "CityWideAverageModelDataset",
                "KRingModelDataset",
                "KRingModelDataset3",
            ]
        ),
        applied_df.test_df.select(
            [
                pl.col("service_time").cast(pl.Float32),
                pl.col("cluster"),
                pl.lit("true").alias("mode"),
            ]
        ),
    )
)

# create figure and axes
fig, ax = plt.subplots()

colors = sns.color_palette("colorblind", 4)

# for i, cluster in enumerate(["pred_", ""]):
sns.violinplot(
    x="cluster",
    y="service_time",
    data=test_df.filter(
        pl.col("service_time") < pl.col("service_time").quantile(0.99)
    ).to_pandas(),
    hue="mode",
    palette=colors,
    split=True,
    ax=ax,
    common_norm=True,
    saturation=0.75,
    inner=None,
)

### Train Everything - Boston, Apply Boston


In [ ]:
import random

model = XGBoostLSSModel()


applied_df = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    .add_van_label()  # this also adds the van label to the feature columns
    .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=[
            "Chicago, USA",
            "Seattle, USA",
            "London, UK",
            "Brussels, Belgium",
        ],
        test_cities=["Boston, USA"],
        min_points_per_hex_train=20,
        min_points_per_hex_test=20,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
    )
    .apply_model(XGBoostLSSModel(), n_boosts=1000)
)

In [ ]:
# score the median prediction
from sklearn.metrics import r2_score

region_test_df = applied_df.test_df.group_by("region_id").agg(pl.all().first())

for q in [0.05, 0.5, 0.95]:
    print(
        q,
        r2_score(
            region_test_df["log_service_time_q{}".format(q)].exp(),
            region_test_df["XGBoostLSS_quant_{}".format(q)].exp(),
        ),
    )

In [ ]:
# import scienceplots
import seaborn as sns
import matplotlib.pyplot as plt

# compute the ks score
from scipy.stats import ks_2samp


# plt.style.use(["science",])
test_df = pl.concat(
    (
        applied_df.test_df.select(
            [
                pl.col("XGBoostLSS_samples").exp().alias("service_time"),
                pl.col("cluster"),
                pl.lit("predicted").alias("mode"),
            ]
        ),
        applied_df.test_df.select(
            [
                pl.col("service_time").cast(pl.Float32),
                pl.col("cluster"),
                pl.lit("empirical").alias("mode"),
            ]
        ),
    )
)


for cluster, cluster_df in test_df.partition_by(["cluster"], as_dict=True).items():
    ks = ks_2samp(
        cluster_df.filter(pl.col("mode") == "empirical")["service_time"],
        cluster_df.filter(pl.col("mode") == "predicted")["service_time"],
    )
    print(f"KS Score for {cluster}: {ks}")

# create figure and axes
fig, ax = plt.subplots()
colors = sns.color_palette("colorblind", 2)

# for i, cluster in enumerate(["pred_", ""]):
sns.violinplot(
    x="cluster",
    y="service_time",
    data=test_df.filter(
        pl.col("service_time") < pl.col("service_time").quantile(0.99)
    ).to_pandas(),
    hue="mode",
    palette=["blue", "red"],
    split=True,
    ax=ax,
    scale_hue=False,
    saturation=0.75,
    inner=None,
)

### Train Bike Model, Apply to Boston


In [ ]:
import random

model = XGBoostLSSModel()


applied_df = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    .add_van_label()  # this also adds the van label to the feature columns
    # .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=[
            "London, UK",
            "Brussels, Belgium",
            "Seattle, USA",
            "Chicago, USA",
        ],
        test_cities=["Boston, USA"],
        min_points_per_hex_train=10,
        min_points_per_hex_test=10,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
    )
    .apply_model(
        XGBoostLSSModel(distribution=Weibull.Weibull()),
        n_boosts=500,
        vehicle_type="cargo_bike",
    )
)

In [ ]:
# import scienceplots
import seaborn as sns
import matplotlib.pyplot as plt

# compute the ks score
from scipy.stats import ks_2samp


# plt.style.use(["science",])
test_df = pl.concat(
    (
        applied_df.test_df.select(
            [
                pl.col("XGBoostLSS_samples").alias("service_time"),
                pl.col("cluster"),
                pl.lit("predicted").alias("mode"),
            ]
        ),
        applied_df.test_df.select(
            [
                pl.col("service_time").cast(pl.Float32),
                pl.col("cluster"),
                pl.lit("empirical").alias("mode"),
            ]
        ),
    )
)


for cluster, cluster_df in test_df.partition_by(["cluster"], as_dict=True).items():
    ks = ks_2samp(
        cluster_df.filter(pl.col("mode") == "empirical")["service_time"],
        cluster_df.filter(pl.col("mode") == "predicted")["service_time"],
    )
    print(f"KS Score for {cluster}: {ks}")

# create figure and axes
fig, ax = plt.subplots()
colors = sns.color_palette("colorblind", 2)

# for i, cluster in enumerate(["pred_", ""]):
sns.violinplot(
    x="cluster",
    y="service_time",
    data=test_df.filter(
        pl.col("service_time") < pl.col("service_time").quantile(0.99)
    ).to_pandas(),
    hue="mode",
    palette=["blue", "red"],
    split=True,
    ax=ax,
    scale_hue=False,
    saturation=0.75,
    inner=None,
)

## Train Van Model, Apply to London


In [ ]:
import random

model = XGBoostLSSModel(distribution=StudentT.StudentT())

applied_df = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    .add_van_label()  # this also adds the van label to the feature columns
    # .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=[
            "Boston, USA",
            "Chicago, USA",
            "Seattle, USA",
            "Brussels, Belgium",
        ],
        test_cities=["London, UK"],
        min_points_per_hex_train=20,
        min_points_per_hex_test=20,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
    )
    .apply_model(
        model,
        n_boosts=500,
        # vehicle_type="van",
    )
)

In [ ]:
import random

model = XGBoostLSSModel(distribution=Weibull.Weibull())

applied_df = (
    ServiceTimeDataset(
        config=load_config(ROOT / "config" / "paper.yaml"),
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    .add_van_label()  # this also adds the van label to the feature columns
    # .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=[
            "Boston, USA",
            "Chicago, USA",
            "Seattle, USA",
            "Brussels, Belgium",
        ],
        test_cities=["London, UK"],
        min_points_per_hex_train=20,
        min_points_per_hex_test=20,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
    )
    .apply_model(
        model,
        n_boosts=500,
        vehicle_type="van",
    )
)

In [ ]:
# applied_df.score_test(
#     models=[model],
# )

In [ ]:
# import scienceplots
import seaborn as sns
import matplotlib.pyplot as plt

# compute the ks score
from scipy.stats import ks_2samp


# plt.style.use(["science",])
test_df = pl.concat(
    (
        applied_df.test_df.select(
            [
                pl.col("XGBoostLSS_samples").alias("service_time"),
                pl.col("cluster"),
                pl.lit("predicted").alias("mode"),
            ]
        ),
        applied_df.test_df.select(
            [
                pl.col("service_time").cast(pl.Float32),
                pl.col("cluster"),
                pl.lit("empirical").alias("mode"),
            ]
        ),
    )
)


for cluster, cluster_df in test_df.partition_by(["cluster"], as_dict=True).items():
    ks = ks_2samp(
        cluster_df.filter(pl.col("mode") == "empirical")["service_time"],
        cluster_df.filter(pl.col("mode") == "predicted")["service_time"],
    )
    print(f"KS Score for {cluster}: {ks}")

# create figure and axes
fig, ax = plt.subplots()
colors = sns.color_palette("colorblind", 2)

# for i, cluster in enumerate(["pred_", ""]):
sns.violinplot(
    x="cluster",
    y="service_time",
    data=test_df.filter(
        pl.col("service_time") < pl.col("service_time").quantile(0.99)
    ).to_pandas(),
    hue="mode",
    palette=["blue", "red"],
    split=True,
    ax=ax,
    scale_hue=False,
    saturation=0.75,
    inner=None,
)

### Trying Different Embeddings


In [ ]:
config = load_config(ROOT / "config" / "paper.yaml")
# config.GeoVex.embedding_file = ROOT / "data/geovex/embeddings_500_randomized.parquet"

applied_df = (
    ServiceTimeDataset(
        config=config,
    )
    .add_cluster_labels(ROOT / "data/geovex/clusters_4.parquet")
    # .set_feature_cols(list(map(str, range(32))))
    .add_van_label()  # this also adds the van label to the feature columns
    .set_pred_col("log_service_time")
    .split_test_train(
        train_cities=[
            "Boston, USA",
            "Brussels, Belgium",
        ],
        test_cities=["London, UK"],
        min_points_per_hex_train=10,
        min_points_per_hex_test=10,
        normalize_test_set_by_total=False,
        seed=random.randint(0, 100000),
        # assertion_override=True,
        # whole_city_test=True,
    )
    .apply_model(XGBoostLSSModel(), n_boosts=500, vehicle_type="van")
)

In [ ]:
# import scienceplots
import seaborn as sns
import matplotlib.pyplot as plt

# compute the ks score
from scipy.stats import ks_2samp


# plt.style.use(["science",])
test_df = pl.concat(
    (
        applied_df.test_df.select(
            [
                pl.col("XGBoostLSS_samples").exp().alias("service_time"),
                pl.col("cluster"),
                pl.lit("predicted").alias("mode"),
            ]
        ),
        applied_df.test_df.select(
            [
                pl.col("service_time").cast(pl.Float32),
                pl.col("cluster"),
                pl.lit("empirical").alias("mode"),
            ]
        ),
    )
)


for cluster, cluster_df in test_df.partition_by(["cluster"], as_dict=True).items():
    ks = ks_2samp(
        cluster_df.filter(pl.col("mode") == "empirical")["service_time"],
        cluster_df.filter(pl.col("mode") == "predicted")["service_time"],
    )
    print(f"KS Score for {cluster}: {ks}")

# create figure and axes
fig, ax = plt.subplots()
colors = sns.color_palette("colorblind", 2)

# for i, cluster in enumerate(["pred_", ""]):
sns.violinplot(
    x="cluster",
    y="service_time",
    data=test_df.filter(
        pl.col("service_time") < pl.col("service_time").quantile(0.99)
    ).to_pandas(),
    hue="mode",
    palette=["blue", "red"],
    split=True,
    ax=ax,
    scale_hue=False,
    saturation=0.75,
    inner=None,
)